In [12]:
import pandas as pd
import datetime
import os
import configparser
from datetime import datetime
import logging
import pandas as pd
from pyspark.sql import SparkSession, HiveContext, Row
from pyspark.sql.functions import udf, col, lit, substring, split, round, when, expr, concat
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, upper, to_date
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import StructType as StructType, DoubleType as DoubleType, StructField as StructField
from pyspark.sql.types import StringType as StringType, IntegerType as IntegerType, TimestampType, DateType

In [13]:
df_airport=pd.read_csv("airport-codes_csv.csv")
df_airport["iso_region"]=df_airport["iso_region"].str.slice(3,6)
df_airport['type'].unique()
df_airport.head(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [14]:
dim_airport_city_state_code=df_airport[['municipality','iso_region']]
dim_airport_city_state_code.columns=['city','state_code']
dim_airport_city_state_code['country']='United States'
dim_airport_city_state_code['city'] = dim_airport_city_state_code['city'].str.upper()
dim_airport_city_state_code.head(5)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,city,state_code,country
0,BENSALEM,PA,United States
1,LEOTI,KS,United States
2,ANCHOR POINT,AK,United States
3,HARVEST,AL,United States
4,NEWPORT,AR,United States


In [4]:
data = {'code':[1,2,3,4,5,6,7],
        'airport_type':['closed','small_airport','medium_airport','large_airport','heliport', 'seaplane_base','balloonport']}
dim_airport_type=pd.DataFrame(data)
dim_airport_type

,code,airport_type
0,1,closed
1,2,small_airport
2,3,medium_airport
3,4,large_airport
4,5,heliport
5,6,seaplane_base
6,7,balloonport


In [5]:
dim_airport=df_airport[["ident","type","name","elevation_ft","municipality","iata_code","coordinates"]]
dim_airport[['longitude','latitude']] = dim_airport['coordinates'].str.split(',',expand=True)
dim_airport=dim_airport.drop(['coordinates'],axis=1)
dim_airport[['longitude','latitude']] = dim_airport[['longitude','latitude']].apply(pd.to_numeric)
dim_airport=dim_airport.round({'latitude': 2, 'longitude': 2})
#dim_airport['longitude']=dim_airport['longitude'].astype(str)+'W'
#dim_airport['latitude']=dim_airport['latitude'].astype(str)+'N'
code=[1,2,3,4,5,6,7]
airport_type=['closed','small_airport','medium_airport','large_airport','heliport', 'seaplane_base','balloonport']
for i in range(len(code)):
    key,val=code[i],airport_type[i]
    dim_airport.loc[(dim_airport['type']==val).squeeze(),'type']=key
dim_airport.columns=['ident','airport_type_code','airport_name','elevation_ft','city','iata_code','airport_longitude','airport_latitude']
dim_airport['city'] = dim_airport['city'].str.upper()
dim_airport.head(5)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,ident,airport_type_code,airport_name,elevation_ft,city,iata_code,airport_longitude,airport_latitude
0,00A,5,Total Rf Heliport,11.0,Bensalem,NaN,-74.93,40.07
1,00AA,2,Aero B Ranch Airport,3435.0,Leoti,NaN,-101.47,38.70
2,00AK,2,Lowell Field,450.0,Anchor Point,NaN,-151.70,59.95
3,00AL,2,Epps Airpark,820.0,Harvest,NaN,-86.77,34.86
4,00AR,1,Newport Hospital & Clinic Heliport,237.0,Newport,NaN,-91.25,35.61


In [97]:
df_immigration=pd.read_csv("immigration_data_sample.csv")
df_immigration['country'] = 'United States'

In [98]:
fact_immigration = df_immigration[['cicid', 'i94yr', 'i94mon', 'i94port', 'i94addr', 'arrdate', 'depdate', 'i94mode','country','matflag','entdepa','entdepd','dtaddto','dtadfile','occup','count']]
fact_immigration.columns = ['cic_id', 'year', 'month', 'city_code', 'state_code', 'arrival_date', 'departure_date', 'mode','country','matflag','entdepa','entdepd','dtaddto','dtadfile','occup','count']
fact_immigration.head(5)

,cic_id,year,month,city_code,state_code,arrival_date,departure_date,mode,country,matflag,entdepa,entdepd,dtaddto,dtadfile,occup,count
0,4084316.0,2016.0,4.0,HHW,HI,20566.0,20573.0,1.0,United States,M,G,O,07202016,20160422,NaN,1.0
1,4422636.0,2016.0,4.0,MCA,TX,20567.0,20568.0,1.0,United States,M,G,R,10222016,20160423,NaN,1.0
2,1195600.0,2016.0,4.0,OGG,FL,20551.0,20571.0,1.0,United States,M,G,O,07052016,20160407,NaN,1.0
3,5291768.0,2016.0,4.0,LOS,CA,20572.0,20581.0,1.0,United States,M,G,O,10272016,20160428,NaN,1.0
4,985523.0,2016.0,4.0,CHM,NY,20550.0,20553.0,3.0,United States,M,Z,K,07042016,20160406,NaN,1.0


In [99]:
data = {'code':[1,2,3,4,5,6,7,8,9,10,11,12,13,14],
        'entdepa':['G','Z','T','O','P','A','K','U','H','F','B','M','N','NAN']}
dim_immigration_entdepa=pd.DataFrame(data)
dim_immigration_entdepa

,code,entdepa
0,1,G
1,2,Z
2,3,T
3,4,O
4,5,P
5,6,A
6,7,K
7,8,U
8,9,H
9,10,F


In [100]:
data = {'code':[1,2,3,4,5,6,7,8,9,10,11,12,13],
        'entdepd':['O','R','K','N','D','I','Q','J','W','L','M','V','NAN']}
dim_immigration_entdepd=pd.DataFrame(data)
dim_immigration_entdepd

,code,entdepd
0,1,O
1,2,R
2,3,K
3,4,N
4,5,D
5,6,I
6,7,Q
7,8,J
8,9,W
9,10,L


In [101]:
map_entdepd={'O':1,'R':2,'K':3,'N':4,'D':5,'I':6,'Q':7,'J':8,'W':9,'L':10,'M':11,'V':12,'NAN':13}
map_entdepa={'G':1,'Z':2,'T':3,'O':4,'P':5,'A':6,'K':7,'U':8,'H':9,'F':10,'B':11,'M':12,'N':13,'NAN':14}

fact_immigration['entdepd'] = fact_immigration['entdepd'].fillna(13)
fact_immigration['entdepa'] = fact_immigration['entdepa'].fillna(14)

fact_immigration=fact_immigration.replace({"entdepd":map_entdepd})
fact_immigration=fact_immigration.replace({"entdepa":map_entdepa})

fact_immigration.columns = ['cic_id', 'year', 'month', 'city_code', 'state_code', 'arrival_date', 'departure_date', 'mode','country','matflag','entdepa_code','entdepd_code','dtaddto','dtadfile','occup','count']

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [102]:
fact_immigration.head(5)

,cic_id,year,month,city_code,state_code,arrival_date,departure_date,mode,country,matflag,entdepa_code,entdepd_code,dtaddto,dtadfile,occup,count
0,4084316.0,2016.0,4.0,HHW,HI,20566.0,20573.0,1.0,United States,M,1,1,07202016,20160422,NaN,1.0
1,4422636.0,2016.0,4.0,MCA,TX,20567.0,20568.0,1.0,United States,M,1,2,10222016,20160423,NaN,1.0
2,1195600.0,2016.0,4.0,OGG,FL,20551.0,20571.0,1.0,United States,M,1,1,07052016,20160407,NaN,1.0
3,5291768.0,2016.0,4.0,LOS,CA,20572.0,20581.0,1.0,United States,M,1,1,10272016,20160428,NaN,1.0
4,985523.0,2016.0,4.0,CHM,NY,20550.0,20553.0,3.0,United States,M,2,3,07042016,20160406,NaN,1.0


In [2]:
import pandas as pd
def to_datetime(date):
    if date is not None:
        return pd.Timestamp('1960-1-1')+pd.to_timedelta(date, unit='D')

def to_datetime_from_mmddyyyy(date,format_str):
    if date is not None:
            date=pd.to_datetime(date,format=format_str,errors='coerce')
            return date      

In [104]:
fact_immigration['dtaddto']=to_datetime_from_mmddyyyy(fact_immigration['dtaddto'],'%m%d%Y')
fact_immigration['dtadfile']=to_datetime_from_mmddyyyy(fact_immigration['dtadfile'],'%Y%m%d')
fact_immigration['arrival_date'] = to_datetime(fact_immigration['arrival_date'])
fact_immigration['departure_date'] = to_datetime(fact_immigration['departure_date'])

In [105]:
fact_immigration.head(5)

,cic_id,year,month,city_code,state_code,arrival_date,departure_date,mode,country,matflag,entdepa_code,entdepd_code,dtaddto,dtadfile,occup,count
0,4084316.0,2016.0,4.0,HHW,HI,2016-04-22,2016-04-29,1.0,United States,M,1,1,2016-07-20,2016-04-22,NaN,1.0
1,4422636.0,2016.0,4.0,MCA,TX,2016-04-23,2016-04-24,1.0,United States,M,1,2,2016-10-22,2016-04-23,NaN,1.0
2,1195600.0,2016.0,4.0,OGG,FL,2016-04-07,2016-04-27,1.0,United States,M,1,1,2016-07-05,2016-04-07,NaN,1.0
3,5291768.0,2016.0,4.0,LOS,CA,2016-04-28,2016-05-07,1.0,United States,M,1,1,2016-10-27,2016-04-28,NaN,1.0
4,985523.0,2016.0,4.0,CHM,NY,2016-04-06,2016-04-09,3.0,United States,M,2,3,2016-07-04,2016-04-06,NaN,1.0


In [106]:
data = {'visa_type':['B1','WT','E2','I1','M2','CPL','E1','M1','F1','GMB','GMT','CP','F2','WB','I','SBP','B2'],
        'visa':[1.0,2.0,1.0,1.0,3.0,2.0,1.0,3.0,3.0,1.0,2.0,2.0,3.0,1.0,1.0,2.0,2.0]}
dim_immigration_visa=pd.DataFrame(data)
dim_immigration_visa

,visa_type,visa
0,B1,1.0
1,WT,2.0
2,E2,1.0
3,I1,1.0
4,M2,3.0
5,CPL,2.0
6,E1,1.0
7,M1,3.0
8,F1,3.0
9,GMB,1.0


In [107]:
dim_immigration_personal = df_immigration[['cicid', 'i94res', 'i94cit', 'biryear', 'gender', 'insnum']]
dim_immigration_personal.columns = ['cic_id', 'country_residence', 'country_citizenship', 'birth_year', 'gender', 'ins_num']
dim_immigration_personal.head(5)

,cic_id,country_residence,country_citizenship,birth_year,gender,ins_num
0,4084316.0,209.0,209.0,1955.0,F,NaN
1,4422636.0,582.0,582.0,1990.0,M,NaN
2,1195600.0,112.0,148.0,1940.0,M,NaN
3,5291768.0,297.0,297.0,1991.0,M,NaN
4,985523.0,111.0,111.0,1997.0,F,NaN


In [108]:
dim_immigration_airline = df_immigration[['cicid', 'airline', 'admnum', 'fltno', 'visatype','visapost']]
dim_immigration_airline.columns = ['cic_id', 'airline', 'admin_num', 'flight_number', 'visa_type','visapost']  
dim_immigration_airline.head(5)

,cic_id,airline,admin_num,flight_number,visa_type,visapost
0,4084316.0,JL,5.658267e+10,00782,WT,NaN
1,4422636.0,*GA,9.436200e+10,XBLNG,B2,MTR
2,1195600.0,LH,5.578047e+10,00464,WT,NaN
3,5291768.0,QR,9.478970e+10,00739,B2,DOH
4,985523.0,NaN,4.232257e+10,LAND,WT,NaN


In [23]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temperature = pd.read_csv(fname)
df_temperature=df_temperature[df_temperature['Country']=='United States']
df_temperature.head(5)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
47555,1820-01-01,2.101,3.217,Abilene,United States,32.95N,100.53W
47556,1820-02-01,6.926,2.853,Abilene,United States,32.95N,100.53W
47557,1820-03-01,10.767,2.395,Abilene,United States,32.95N,100.53W
47558,1820-04-01,17.989,2.202,Abilene,United States,32.95N,100.53W
47559,1820-05-01,21.809,2.036,Abilene,United States,32.95N,100.53W


In [24]:
dim_temperature_city=df_temperature[['City','Latitude','Longitude']]
dim_temperature_city.drop_duplicates(subset=['City'],keep='first',inplace = True)
dim_temperature_city.reset_index(drop=True, inplace=True)
dim_temperature_city.loc[dim_temperature_city['Latitude'].str[-1:]=='N','Latitude']=dim_temperature_city['Latitude'].str[:-1]
dim_temperature_city.loc[dim_temperature_city['Latitude'].str[-1:]=='S','Latitude']='-'+dim_temperature_city['Latitude'].str[:-1]
dim_temperature_city.loc[dim_temperature_city['Longitude'].str[-1:]=='W','Longitude']='-'+dim_temperature_city['Longitude'].str[:-1]
dim_temperature_city.loc[dim_temperature_city['Longitude'].str[-1:]=='E','Longitude']=dim_temperature_city['Longitude'].str[:-1]
dim_temperature_city.columns=['city','latitude','longitude']
dim_temperature_city['city'] = dim_temperature_city['city'].str.upper()
dim_temperature_city.head(5)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWit

,city,latitude,longitude
0,ABILENE,32.95,-100.53
1,AKRON,40.99,-80.95
2,ALBUQUERQUE,34.56,-107.03
3,ALEXANDRIA,39.38,-76.99
4,ALLENTOWN,40.99,-74.56


In [17]:
dim_temperature_usa=df_temperature[['dt','AverageTemperature','AverageTemperatureUncertainty','City','Country']].reset_index(drop=True)
dim_temperature_usa['dt'] = pd.to_datetime(dim_temperature_usa['dt'])
dim_temperature_usa['year'] = dim_temperature_usa['dt'].apply(lambda t: t.year)
dim_temperature_usa['month'] = dim_temperature_usa['dt'].apply(lambda t: t.month)
dim_temperature_usa.columns=['date','avg_temp','avg_temp_var','city_code','country','year','month']
dim_temperature_usa.head(5)

,date,avg_temp,avg_temp_var,city_code,country,year,month
0,1820-01-01,2.101,3.217,Abilene,United States,1820,1
1,1820-02-01,6.926,2.853,Abilene,United States,1820,2
2,1820-03-01,10.767,2.395,Abilene,United States,1820,3
3,1820-04-01,17.989,2.202,Abilene,United States,1820,4
4,1820-05-01,21.809,2.036,Abilene,United States,1820,5


In [18]:
dim_temperature_usa_2014 = dim_temperature_usa[dim_temperature_usa['year'] == 2014]
dim_temperature_usa_2014.head(5)

,date,avg_temp,avg_temp_var,city_code,country,year,month


In [49]:
df_demographics = pd.read_csv('us-cities-demographics.csv', delimiter=';')
df_demographics.head(5)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [50]:
df_demographics['Race'].unique()

array(['Hispanic or Latino', 'White', 'Asian', 'Black or African-American',
       'American Indian and Alaska Native'], dtype=object)

In [51]:
data={'race_code':[1,2,3,4,5],
      'race_type':['Hispanic or Latino','White','Asian','Black or African-American','American Indian and Alaska Native']
     }
dim_demographics_race=pd.DataFrame(data)
dim_demographics_race

,race_code,race_type
0,1,Hispanic or Latino
1,2,White
2,3,Asian
3,4,Black or African-American
4,5,American Indian and Alaska Native


In [52]:
#city_code=[i for i in range(1,597)]
dim_demographics_city_population = df_demographics[['City','State Code','Male Population','Female Population','Number of Veterans','Foreign-born','Median Age', 'Average Household Size']].drop_duplicates()
dim_demographics_city_population.columns = ['city','state_code','male_pop','female_pop','num_vetarans','foreign_born','median_age', 'avg_household_size']
#dim_demographics_city_population.insert(loc=0, column='city_code', value=city_code)
dim_demographics_city_population['idx']=dim_demographics_city_population['city']+dim_demographics_city_population['state_code']
dim_demographics_city_population['city'] = dim_demographics_city_population['city'].str.upper()
dim_demographics_city_population['idx'] = dim_demographics_city_population['idx'].str.upper()
dim_demographics_city_population.head(5)

,city,state_code,male_pop,female_pop,num_vetarans,foreign_born,median_age,avg_household_size,idx
0,SILVER SPRING,MD,40601.0,41862.0,1562.0,30908.0,33.8,2.60,SILVER SPRINGMD
1,QUINCY,MA,44129.0,49500.0,4147.0,32935.0,41.0,2.39,QUINCYMA
2,HOOVER,AL,38040.0,46799.0,4819.0,8229.0,38.5,2.58,HOOVERAL
3,RANCHO CUCAMONGA,CA,88127.0,87105.0,5821.0,33878.0,34.5,3.18,RANCHO CUCAMONGACA
4,NEWARK,NJ,138040.0,143873.0,5829.0,86253.0,34.6,2.73,NEWARKNJ


In [53]:
dim_demographics_city_race=df_demographics[['City','State Code','Race']]
dim_demographics_city_race['idx']=dim_demographics_city_race['City']+dim_demographics_city_race['State Code']
dim_demographics_city_race=dim_demographics_city_race[['idx','Race']]
dim_demographics_city_race['idx'] = dim_demographics_city_race['idx'].str.upper()
#dim_demographics_city_population=dim_demographics_city_population.drop(columns=['City','State Code'],axis=1)
#dim_demographics_city_population=dim_demographics_city_population.drop(columns=['State Code'])
dim_demographics_city_race.head(5)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,idx,Race
0,SILVER SPRINGMD,Hispanic or Latino
1,QUINCYMA,White
2,HOOVERAL,Asian
3,RANCHO CUCAMONGACA,Black or African-American
4,NEWARKNJ,White


In [54]:
#dim_demographics_city_race['City'] = dim_demographics_city_race['idx'].map(dim_demographics_city_population.set_index('idx')['city_code'])
dim_demographics_city_race['Race'] = dim_demographics_city_race['Race'].map(dim_demographics_race.set_index('race_type')['race_code'])
#dim_demographics_city_race=dim_demographics_city_race.drop(columns=['State Code','idx'])
dim_demographics_city_race.columns=['idx','race_code']
dim_demographics_city_race['idx'] = dim_demographics_city_race['idx'].str.upper()
#dim_demographics_city_population=dim_demographics_city_population.drop(columns=['idx'])
dim_demographics_city_race.head(5)

,idx,race_code
0,SILVER SPRINGMD,1
1,QUINCYMA,2
2,HOOVERAL,3
3,RANCHO CUCAMONGACA,4
4,NEWARKNJ,2


In [19]:
with open("I94_SAS_Labels_Descriptions.SAS") as f:
    contents = f.readlines()

In [20]:
country_code = {}
for countries in contents[10:298]:
    pair = countries.split('=')
    code, country = pair[0].strip(), pair[1].strip().strip("'")
    country_code[code] = country
df_country_code = pd.DataFrame(list(country_code.items()), columns=['code', 'country'])
df_country_code.head(5)    

,code,country
0,236,AFGHANISTAN
1,101,ALBANIA
2,316,ALGERIA
3,102,ANDORRA
4,324,ANGOLA


In [21]:
city_code = {}
for cities in contents[303:962]:
    pair = cities.split('=')
    code, city = pair[0].strip("\t").strip().strip("'"), pair[1].strip('\t').strip().strip("''")
    city_code[code] = city
df_city_code = pd.DataFrame(list(city_code.items()), columns=['abbr_code', 'city'])
df_city_code.head(5)    

,abbr_code,city
0,ANC,"ANCHORAGE, AK"
1,BAR,"BAKER AAF - BAKER ISLAND, AK"
2,DAC,"DALTONS CACHE, AK"
3,PIZ,"DEW STATION PT LAY DEW, AK"
4,DTH,"DUTCH HARBOR, AK"


In [22]:
state_code = {}
for states in contents[982:1036]:
    pair = states.split('=')
    code, state = pair[0].strip('\t').strip("'"), pair[1].strip().strip("'")
    state_code[code] = state
df_state_code = pd.DataFrame(list(state_code.items()), columns=['abbr_code', 'state'])
df_state_code.head(5)    

,abbr_code,state
0,AK,ALASKA
1,AZ,ARIZONA
2,AR,ARKANSAS
3,CA,CALIFORNIA
4,CO,COLORADO


In [ ]:
dim_demographics_city_population['city'] = dim_demographics_city_population['city'].str.upper()
dim_demographics_city_population.head(5)

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lit

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()
df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [3]:
df_spark = df_spark.withColumn('country', lit('United States'))

In [4]:
from pyspark.sql.functions import monotonically_increasing_id
fact_immigration = df_spark.select('cicid', 'i94yr', 'i94mon', 'i94port', 'i94addr',\
                                   'arrdate', 'depdate', 'i94mode', 'country', 'matflag',\
                                   'entdepa','entdepd','dtaddto','dtadfile','occup',\
                                   'count').distinct()\
.withColumn("immigration_id", monotonically_increasing_id())

In [5]:
fact_immigration.show(5)

+-----+------+------+-------+-------+-------+-------+-------+-------------+-------+-------+-------+--------+--------+-----+-----+--------------+
|cicid| i94yr|i94mon|i94port|i94addr|arrdate|depdate|i94mode|      country|matflag|entdepa|entdepd| dtaddto|dtadfile|occup|count|immigration_id|
+-----+------+------+-------+-------+-------+-------+-------+-------------+-------+-------+-------+--------+--------+-----+-----+--------------+
|128.0|2016.0|   4.0|    NEW|     NJ|20545.0|20552.0|    1.0|United States|      M|      G|      Q|06292016|20160401| null|  1.0|             0|
|296.0|2016.0|   4.0|    NYC|     NY|20545.0|20548.0|    1.0|United States|      M|      O|      O|06292016|20160401| null|  1.0|             1|
|520.0|2016.0|   4.0|    CHI|     TN|20545.0|20577.0|    1.0|United States|      M|      G|      O|06292016|20160401| null|  1.0|             2|
|901.0|2016.0|   4.0|    NEW|     NV|20545.0|20549.0|    1.0|United States|      M|      G|      O|06292016|20160401| null|  1.0| 

In [5]:
def update_column_name(original_table, new_column_names):
    for original_column_name, new_column_name in zip(original_table.columns, new_column_names):
        original_table = original_table.withColumnRenamed(original_column_name, new_column_name)
    return original_table

In [7]:
    new_columns = ['cic_id', 'year', 'month', 'city_code', 'state_code', 'arrival_date',\
                   'departure_date','mode','country','matflag','entdepa','entdepd',\
                   'dtaddto','dtadfile','occup','count','immigration_id']
                  
    fact_immigration = update_column_name(fact_immigration, new_columns)

In [8]:
fact_immigration.show(5)

+------+------+-----+---------+----------+------------+--------------+----+-------------+-------+-------+-------+--------+--------+-----+-----+--------------+
|cic_id|  year|month|city_code|state_code|arrival_date|departure_date|mode|      country|matflag|entdepa|entdepd| dtaddto|dtadfile|occup|count|immigration_id|
+------+------+-----+---------+----------+------------+--------------+----+-------------+-------+-------+-------+--------+--------+-----+-----+--------------+
| 128.0|2016.0|  4.0|      NEW|        NJ|     20545.0|       20552.0| 1.0|United States|      M|      G|      Q|06292016|20160401| null|  1.0|             0|
| 296.0|2016.0|  4.0|      NYC|        NY|     20545.0|       20548.0| 1.0|United States|      M|      O|      O|06292016|20160401| null|  1.0|             1|
| 520.0|2016.0|  4.0|      CHI|        TN|     20545.0|       20577.0| 1.0|United States|      M|      G|      O|06292016|20160401| null|  1.0|             2|
| 901.0|2016.0|  4.0|      NEW|        NV|    

In [9]:
from pyspark.sql.functions import udf, col, lit, year, month, upper, to_date
maps_entdepd={'O':'1','R':'2','K':'3','N':'4','D':'5','I':'6','Q':'7','J':'8','W':'9','L':'10','M':'11','V':'12','null':'13'}
fact_immigration = fact_immigration.replace(to_replace=maps_entdepd, subset=['entdepd'])
fact_immigration=fact_immigration.na.fill({'entdepd': '13'})
fact_immigration.withColumn('entdepd',col('entdepd').cast('int'))
fact_immigration.select('entdepd').distinct().show()

+-------+
|entdepd|
+-------+
|      7|
|     11|
|      3|
|      8|
|      5|
|      6|
|      9|
|      1|
|     10|
|      4|
|     12|
|     13|
|      2|
+-------+



In [11]:
data = [{'code' : 1, 'entdepa' : 'G'}, {'code' : 2, 'entdepa' : 'Z'},
        {'code' : 3, 'entdepa' : 'T'}, {'code' : 4, 'entdepa' : 'O'},
        {'code' : 5, 'entdepa' : 'P'}, {'code' : 6, 'entdepa' : 'A'},
        {'code' : 7, 'entdepa' : 'K'}, {'code' : 8, 'entdepa' : 'U'},
        {'code' : 9, 'entdepa' : 'H'}, {'code' : 10, 'entdepa' : 'F'},
        {'code' : 11, 'entdepa' : 'B'}, {'code' : 12, 'entdepa' : 'M'},
        {'code' : 13, 'entdepa' : 'N'}, {'code' : 14, 'entdepa' : 'null'}]

dim_immigration_entdepa = spark.createDataFrame(data)
dim_immigration_entdepa.write.mode("overwrite")\
.parquet(path='immigration_data/dim_immigration_entdepa') 

/opt/spark-2.4.3-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [12]:
data = [{'code' : 1, 'entdepd' : 'O'}, {'code' : 2, 'entdepd' : 'R'},
        {'code' : 3, 'entdepd' : 'K'}, {'code' : 4, 'entdepd' : 'N'},
        {'code' : 5, 'entdepd' : 'D'}, {'code' : 6, 'entdepd' : 'I'},
        {'code' : 7, 'entdepd' : 'Q'}, {'code' : 8, 'entdepd' : 'J'},
        {'code' : 9, 'entdepd' : 'W'}, {'code' : 10, 'entdepd' : 'L'},
        {'code' : 11, 'entdepd' : 'M'}, {'code' : 12, 'entdepd' : 'V'},
        {'code' : 13, 'entdepd' : 'null'}]

dim_immigration_entdepd = spark.createDataFrame(data)
dim_immigration_entdepd.write.mode("overwrite")\
.parquet(path='immigration_data/dim_immigration_entdepd')

/opt/spark-2.4.3-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [13]:
maps_entdepa={'G':'1','Z':'2','T':'3','O':'4','P':'5','A':'6','K':'7','U':'8','H':'9','F':'10','B':'11','M':'12','N':'13','null':'14'}
fact_immigration = fact_immigration.replace(to_replace=maps_entdepa, subset=['entdepa'])
fact_immigration=fact_immigration.na.fill({'entdepa': '14'})
fact_immigration.withColumn('entdepa',col('entdepa').cast('int'))
fact_immigration.select('entdepa').distinct().show()

+-------+
|entdepa|
+-------+
|      7|
|     11|
|      3|
|      8|
|      5|
|      6|
|      9|
|      1|
|     10|
|      4|
|     12|
|     13|
|     14|
|      2|
+-------+



In [14]:
data = [{'visa_type' : 'B1', 'visa' : 1.0}, {'visa_type' : 'WT', 'visa' : 2.0},
        {'visa_type' : 'E2', 'visa' : 1.0}, {'visa_type' : 'I1', 'visa' : 1.0},
        {'visa_type' : 'M2', 'visa' : 3.0}, {'visa_type' : 'CPL', 'visa' : 2.0},
        {'visa_type' : 'E1', 'visa' : 1.0}, {'visa_type' : 'M1', 'visa' : 3.0},
        {'visa_type' : 'F1', 'visa' : 3.0}, {'visa_type' : 'GMB', 'visa' : 1.0},
        {'visa_type' : 'GMT', 'visa' : 2.0}, {'visa_type' : 'CP', 'visa' : 2.0},
        {'visa_type' : 'F2', 'visa' : 3.0}, {'visa_type' : 'WB', 'visa' : 1.0},
        {'visa_type' : 'I', 'visa' : 1.0}, {'visa_type' : 'SBP', 'visa' : 2.0},
        {'visa_type' : 'B2', 'visa' : 2.0}]
dim_immigration_visa = spark.createDataFrame(data)
dim_immigration_visa = dim_immigration_visa.select(sorted(dim_immigration_visa.columns,reverse=True))
dim_immigration_visa.write.mode("overwrite")\
.parquet(path='immigration_data/dim_immigration_visa')

/opt/spark-2.4.3-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [15]:
import os
import configparser
from datetime import datetime
import logging
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lit
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, upper, to_date
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import StructType as StructType, DoubleType as DoubleType, StructField as StructField
from pyspark.sql.types import StringType as StringType, IntegerType as IntegerType, TimestampType, DateType

def convert_to_datetime(date):
    """
    Convert to yyyy-mm-dd format
    
    :return: date in yyyy-mm-dd format
    """   
    if date is not None:
        return pd.Timestamp('1960-1-1')+pd.to_timedelta(date, unit='D')

convert_to_datetime_udf = udf(convert_to_datetime, DateType())

def convert_to_datetime_from_format(format_str):
    def trans(date):
        if date is not None:
            date=pd.to_datetime(date,format=format_str,errors='coerce')
            return date      
    return udf(trans,DateType())

In [16]:
fact_immigration = fact_immigration.withColumn('arrival_date', \
                               convert_to_datetime_udf(col('arrival_date')))
fact_immigration = fact_immigration.withColumn('departure_date', \
                               convert_to_datetime_udf(col('departure_date')))
#df_spark = df_spark.withColumn('dtaddto', \
#                               convert_to_datetime_from_format('%m%d%Y')(col('dtaddto')))
#df_spark = df_spark.withColumn('dtadfile', \
#                               convert_to_datetime_from_format('%Y%m%d')(col('dtadfile')))                 

In [17]:
fact_immigration = fact_immigration.withColumn('dtadfile', \
                               convert_to_datetime_from_format('%Y%m%d')(col('dtadfile'))) 
#fact_immigration.select('dtadfile').show(5)

In [18]:
from pyspark.sql.functions import regexp_replace
fact_immigration=fact_immigration.withColumn('dtaddto', regexp_replace('dtaddto', 'D/S', '01011900'))
#fact_immigration.select('dtaddto').show(10)

In [19]:
fact_immigration = fact_immigration.withColumn('dtaddto', \
                               convert_to_datetime_from_format('%m%d%Y')(col('dtaddto'))) 
#fact_immigration.select('dtaddto').show(10)

In [20]:
    new_columns = ['cic_id', 'year', 'month', 'city_code', 'state_code', 'arrival_date',\
                   'departure_date','mode','country','matflag','entdepa_code',\
                   'entdepd_code','dtaddto','dtadfile','occup','count','immigration_id']
                  
    fact_immigration = update_column_name(fact_immigration, new_columns)
#fact_immigration.show(1)

In [21]:
#fact_immigration=fact_immigration.na.fill({'state_code': 'XXXXX'})
#fact_immigration.write.mode("overwrite").parquet(path='fact_immigration') 
#fact_immigration.select('state_code').distinct().show(5)
fact_immigration = fact_immigration.filter("mode is not NULL AND occup is not NULL")
fact_immigration.write.mode("overwrite").partitionBy('state_code')\
.parquet(path='immigration_data/fact_immigration') 
#fact_immigration.write.mode("overwrite").parquet(path='fact_immigration')

In [22]:
dim_immigration_airline = df_spark.select('cicid', 'airline', 'admnum', 'fltno',\
                                          'visatype','visapost').distinct()\
.withColumn("immigration_airline_id", monotonically_increasing_id())
new_columns = ['cic_id', 'airline', 'admin_num', 'flight_number', 'visa_type',\
               'visapost', 'immigration_airline_id']
                  
dim_immigration_airline = update_column_name(dim_immigration_airline, new_columns)
dim_immigration_airline.write.mode("overwrite")\
.parquet(path='immigration_data/dim_immigration_airline')

In [23]:
dim_immigration_personal = df_spark.select('cicid', 'i94cit', 'i94res', 'biryear',\
                                          'gender', 'insnum').distinct()\
.withColumn("immi_personal_id", monotonically_increasing_id())
new_columns = ['cic_id', 'country_residence','country_citizenship',\
               'birth_year', 'gender', 'ins_num', 'immi_personal_id']
                  
dim_immigration_personal = update_column_name(dim_immigration_personal, new_columns)
dim_immigration_personal.write.mode("overwrite")\
.parquet(path='immigration_data/dim_immigration_personal')

In [71]:
df_spark=spark.read.parquet("sas_data")
df_spark.head(1)

[Row(cicid=5748517.0, i94yr=2016.0, i94mon=4.0, i94cit=245.0, i94res=438.0, i94port='LOS', arrdate=20574.0, i94mode=1.0, i94addr='CA', depdate=20582.0, i94bir=40.0, i94visa=1.0, count=1.0, dtadfile='20160430', visapost='SYD', occup=None, entdepa='G', entdepd='O', entdepu=None, matflag='M', biryear=1976.0, dtaddto='10292016', gender='F', insnum=None, airline='QF', admnum=94953870030.0, fltno='00011', visatype='B1')]

In [24]:
#df_airport = spark.read.format('com.github.saurfang.sas.spark').load('airport-codes_csv.csv')
#df_airport  = spark.read.format("csv").option("header", "true").load('airport-codes_csv.csv')
df_airport  = spark.read.csv('airport-codes_csv.csv', header=True)

In [25]:
from pyspark.sql.functions import substring
df_airport=df_airport.withColumn('iso_region', substring('iso_region', 4,5))
df_airport.show()

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|        PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|        KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|        AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|     

In [26]:
df_airport.select('type').distinct().show()

+--------------+
|          type|
+--------------+
| large_airport|
|   balloonport|
| seaplane_base|
|      heliport|
|        closed|
|medium_airport|
| small_airport|
+--------------+



In [27]:
data = [{'code' : 1, 'airport_type' : 'closed'}, {'code' : 2, 'airport_type' : 'small_airport'},
        {'code' : 3, 'airport_type' : 'medium_airport'}, {'code' : 4, 'airport_type' : 'large_airport'},
        {'code' : 5, 'airport_type' : 'heliport'}, {'code' : 6, 'airport_type' : 'seaplane_base'},
        {'code' : 7, 'airport_type' : 'balloonport'}]

dim_airport_type = spark.createDataFrame(data)

/opt/spark-2.4.3-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [28]:
dim_airport_type = dim_airport_type.select(sorted(dim_airport_type.columns,reverse=True))
dim_airport_type = dim_airport_type.withColumn('code',col('code').cast("int"))
dim_airport_type.show()
dim_airport_type.write.mode("overwrite").parquet(path='airport_data/dim_airport_type')

+----+--------------+
|code|  airport_type|
+----+--------------+
|   1|        closed|
|   2| small_airport|
|   3|medium_airport|
|   4| large_airport|
|   5|      heliport|
|   6| seaplane_base|
|   7|   balloonport|
+----+--------------+



In [29]:
dim_airport_city_state_code = df_airport.select(['municipality','iso_region'])
dim_airport_city_state_code = dim_airport_city_state_code.withColumn('country', lit('United States'))
new_columns = ['city', 'state_code', 'country']
dim_airport_city_state_code = update_column_name(dim_airport_city_state_code, new_columns)
dim_airport_city_state_code = dim_airport_city_state_code.withColumn('city', upper(col('city')))
dim_airport_city_state_code.write.mode("overwrite").parquet(path='airport_data/dim_airport_city_state_code')
dim_airport_city_state_code.show(5)

+------------+----------+-------------+
|        city|state_code|      country|
+------------+----------+-------------+
|    BENSALEM|        PA|United States|
|       LEOTI|        KS|United States|
|ANCHOR POINT|        AK|United States|
|     HARVEST|        AL|United States|
|     NEWPORT|        AR|United States|
+------------+----------+-------------+
only showing top 5 rows



In [31]:
dim_airport = df_airport.select(["ident","type","name","elevation_ft","municipality","iata_code","coordinates"])
dim_airport.show(5)

+-----+-------------+--------------------+------------+------------+---------+--------------------+
|ident|         type|                name|elevation_ft|municipality|iata_code|         coordinates|
+-----+-------------+--------------------+------------+------------+---------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|    Bensalem|     null|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       Leoti|     null|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|Anchor Point|     null|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|     Harvest|     null|-86.7703018188476...|
| 00AR|       closed|Newport Hospital ...|         237|     Newport|     null| -91.254898, 35.6087|
+-----+-------------+--------------------+------------+------------+---------+--------------------+
only showing top 5 rows



In [32]:
from pyspark.sql.functions import split
split_col = split(dim_airport['coordinates'], ',')
dim_airport = dim_airport.withColumn('airport_longitude', split_col.getItem(0))
dim_airport = dim_airport.withColumn('airport_latitude', split_col.getItem(1))
dim_airport = dim_airport.drop("coordinates")
dim_airport.withColumn('airport_longitude',col('airport_longitude').cast("float"))
dim_airport.withColumn('airport_latitude',col('airport_latitude').cast("float"))
dim_airport.show(5)

+-----+-------------+--------------------+------------+------------+---------+------------------+------------------+
|ident|         type|                name|elevation_ft|municipality|iata_code| airport_longitude|  airport_latitude|
+-----+-------------+--------------------+------------+------------+---------+------------------+------------------+
|  00A|     heliport|   Total Rf Heliport|          11|    Bensalem|     null|-74.93360137939453|    40.07080078125|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       Leoti|     null|       -101.473911|         38.704022|
| 00AK|small_airport|        Lowell Field|         450|Anchor Point|     null|    -151.695999146|       59.94919968|
| 00AL|small_airport|        Epps Airpark|         820|     Harvest|     null|-86.77030181884766| 34.86479949951172|
| 00AR|       closed|Newport Hospital ...|         237|     Newport|     null|        -91.254898|           35.6087|
+-----+-------------+--------------------+------------+---------

In [33]:
from pyspark.sql.functions import round
dim_airport = dim_airport.withColumn('airport_longitude', round(dim_airport['airport_longitude'], 2))
dim_airport = dim_airport.withColumn('airport_latitude', round(dim_airport['airport_latitude'], 2))
dim_airport.show(5)

+-----+-------------+--------------------+------------+------------+---------+-----------------+----------------+
|ident|         type|                name|elevation_ft|municipality|iata_code|airport_longitude|airport_latitude|
+-----+-------------+--------------------+------------+------------+---------+-----------------+----------------+
|  00A|     heliport|   Total Rf Heliport|          11|    Bensalem|     null|           -74.93|           40.07|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       Leoti|     null|          -101.47|            38.7|
| 00AK|small_airport|        Lowell Field|         450|Anchor Point|     null|           -151.7|           59.95|
| 00AL|small_airport|        Epps Airpark|         820|     Harvest|     null|           -86.77|           34.86|
| 00AR|       closed|Newport Hospital ...|         237|     Newport|     null|           -91.25|           35.61|
+-----+-------------+--------------------+------------+------------+---------+----------

In [34]:
code=[1,2,3,4,5,6,7]
maps_airport_type = {'closed':'1','small_airport':'2','medium_airport':'3','large_airport':'4','heliport':'5', 'seaplane_base':'6','balloonport':'7'}
dim_airport = dim_airport.replace(to_replace=maps_airport_type, subset=['type'])
new_columns  = ['ident','airport_type_code','airport_name','elevation_ft','city','iata_code','airport_longitude','airport_latitude']
dim_airport = update_column_name(dim_airport, new_columns)
dim_airport = dim_airport.withColumn('airport_type_code',col('airport_type_code').cast("int"))
dim_airport = dim_airport.withColumn('city', upper(col('city')))
dim_airport.write.mode("overwrite").parquet(path='airport_data/dim_airport')
dim_airport.show(5)

+-----+-----------------+--------------------+------------+------------+---------+-----------------+----------------+
|ident|airport_type_code|        airport_name|elevation_ft|        city|iata_code|airport_longitude|airport_latitude|
+-----+-----------------+--------------------+------------+------------+---------+-----------------+----------------+
|  00A|                5|   Total Rf Heliport|          11|    BENSALEM|     null|           -74.93|           40.07|
| 00AA|                2|Aero B Ranch Airport|        3435|       LEOTI|     null|          -101.47|            38.7|
| 00AK|                2|        Lowell Field|         450|ANCHOR POINT|     null|           -151.7|           59.95|
| 00AL|                2|        Epps Airpark|         820|     HARVEST|     null|           -86.77|           34.86|
| 00AR|                1|Newport Hospital ...|         237|     NEWPORT|     null|           -91.25|           35.61|
+-----+-----------------+--------------------+----------

In [35]:
df_temperature  = spark.read.csv('../../data2/GlobalLandTemperaturesByCity.csv', header=True)
df_temperature = df_temperature.where(df_temperature['Country'] == 'United States')
df_temperature.show(5)

+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|   City|      Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|1820-01-01|2.1010000000000004|                        3.217|Abilene|United States|  32.95N|  100.53W|
|1820-02-01|             6.926|                        2.853|Abilene|United States|  32.95N|  100.53W|
|1820-03-01|            10.767|                        2.395|Abilene|United States|  32.95N|  100.53W|
|1820-04-01|17.988999999999994|                        2.202|Abilene|United States|  32.95N|  100.53W|
|1820-05-01|            21.809|                        2.036|Abilene|United States|  32.95N|  100.53W|
+----------+------------------+-----------------------------+-------+-------------+--------+---------+
only showing top 5 rows



In [36]:
dim_temperature_city = df_temperature.select(['City','Latitude','Longitude']).distinct() 
dim_temperature_city.show(5)

+------------+--------+---------+
|        City|Latitude|Longitude|
+------------+--------+---------+
| New Orleans|  29.74N|   90.46W|
| Kansas City|  39.38N|   93.64W|
| Springfield|  37.78N|   93.56W|
|Independence|  39.38N|   93.64W|
|     Fontana|  34.56N|  116.76W|
+------------+--------+---------+
only showing top 5 rows



In [37]:
from pyspark.sql.functions import when, substring, expr

dim_temperature_city = dim_temperature_city.withColumn('Latitude', when(substring('Latitude', -1,1)=='N',\
                                                                        expr("substring(Latitude, 1, length(Latitude)-1)")).otherwise\
                                                       (-expr("substring(Latitude, 1, length(Latitude)-1)")))

dim_temperature_city = dim_temperature_city.withColumn('Longitude', when(substring('Longitude', -1,1)=='E',\
                                                                        expr("substring(Longitude, 1, length(Longitude)-1)")).otherwise\
                                                       (-expr("substring(Longitude, 1, length(Longitude)-1)")))

new_columns=['city','latitude','longitude']
dim_temperature_city = update_column_name(dim_temperature_city, new_columns)
dim_temperature_city = dim_temperature_city.withColumn('city', upper(col('city')))
dim_temperature_city.show(5)

+------------+--------+---------+
|        city|latitude|longitude|
+------------+--------+---------+
| NEW ORLEANS|   29.74|   -90.46|
| KANSAS CITY|   39.38|   -93.64|
| SPRINGFIELD|   37.78|   -93.56|
|INDEPENDENCE|   39.38|   -93.64|
|     FONTANA|   34.56|  -116.76|
+------------+--------+---------+
only showing top 5 rows



In [38]:
dim_temperature_city.write.mode("overwrite").parquet(path='temperature_data/dim_temperature_city')

In [100]:
dim_temperature_usa=df_temperature.select(['dt','AverageTemperature','AverageTemperatureUncertainty','City','Country']).distinct()
dim_temperature_usa = dim_temperature_usa.withColumn('year', substring('dt',1,4))
dim_temperature_usa = dim_temperature_usa.withColumn('month', substring(substring('dt',6,6),1,2))
new_columns = ['date','avg_temp','avg_temp_var','city','country','year','month']
dim_temperature_usa = update_column_name(dim_temperature_usa, new_columns)
dim_temperature_usa = dim_temperature_usa.withColumn('year',col('year').cast("int"))
dim_temperature_usa = dim_temperature_usa.withColumn('month',col('month').cast("int"))
dim_temperature_usa.write.mode("overwrite").parquet(path='temperature_data/dim_temperature_usa')
dim_temperature_usa.show(5)

+----------+------------------+------------------+-------+-------------+----+-----+
|      date|          avg_temp|      avg_temp_var|   city|      country|year|month|
+----------+------------------+------------------+-------+-------------+----+-----+
|1821-06-01|            25.768|             2.653|Abilene|United States|1821|    6|
|1830-11-01|            13.302|             2.715|Abilene|United States|1830|   11|
|1836-11-01|             8.827|2.1719999999999997|Abilene|United States|1836|   11|
|1846-07-01|28.258000000000003|2.0069999999999997|Abilene|United States|1846|    7|
|1863-04-01|            18.553|1.1740000000000002|Abilene|United States|1863|    4|
+----------+------------------+------------------+-------+-------------+----+-----+
only showing top 5 rows



In [39]:
df_demographics = spark.read.format('csv').options(header=True, delimiter=';').load('us-cities-demographics.csv')
df_demographics.show(5)

+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|            City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race|Count|
+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|   Silver Spring|     Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino|25924|
|          Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|               White|58723|
|          Hoover|      Alabama|      38.5|          38040| 

In [40]:
data = [{'race_code' : 1, 'race_type' : 'Hispanic or Latino'}, {'race_code' : 2, 'race_type' : 'White'},
        {'race_code' : 3, 'race_type' : 'Asian'}, {'race_code' : 4, 'race_type' : 'Black or African-American'},
        {'race_code' : 5, 'race_type' : 'American Indian and Alaska Native'}]

dim_demographics_race = spark.createDataFrame(data)
dim_demographics_race.write.mode("overwrite")\
.parquet(path='demographics_data/dim_demographics_race')
dim_demographics_race.show()

/opt/spark-2.4.3-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+---------+--------------------+
|race_code|           race_type|
+---------+--------------------+
|        1|  Hispanic or Latino|
|        2|               White|
|        3|               Asian|
|        4|Black or African-...|
|        5|American Indian a...|
+---------+--------------------+



In [43]:
maps_race_type = {'Hispanic or Latino':'1','White':'2','Asian':'3','Black or African-American':'4',\
                  'American Indian and Alaska Native':'5'}
dim_demographics_city_race = df_demographics.select(['City','State Code','Race'])
new_columns = ['city','state','race']
dim_demographics_city_race = update_column_name(dim_demographics_city_race, new_columns)
dim_demographics_city_race = dim_demographics_city_race.select(concat(dim_demographics_city_race.city,\
                                                                      dim_demographics_city_race.state)\
                                                               .alias('idx'),'race')
dim_demographics_city_race = dim_demographics_city_race.replace(to_replace=maps_race_type, subset=['race'])
new_columns = ['idx','race_code']
dim_demographics_city_race = update_column_name(dim_demographics_city_race, new_columns)
dim_demographics_city_race = dim_demographics_city_race\
.withColumn('idx', upper(col('idx')))
dim_demographics_city_race.write.mode("overwrite")\
.parquet(path='demographics_data/dim_demographics_city_race')
dim_demographics_city_race.show(5)

+------------------+---------+
|               idx|race_code|
+------------------+---------+
|   SILVER SPRINGMD|        1|
|          QUINCYMA|        2|
|          HOOVERAL|        3|
|RANCHO CUCAMONGACA|        4|
|          NEWARKNJ|        2|
+------------------+---------+
only showing top 5 rows



In [44]:
    dim_demographics_city_population = df_demographics.select(['City','State Code','Male Population',\
                                                               'Female Population','Number of Veterans',\
                                                               'Foreign-born','Median Age',\
                                                               'Average Household Size']).distinct()
    new_columns = ['city','state_code','male_pop','female_pop','num_vetarans','foreign_born',\
                   'median_age','avg_household_size']
    dim_demographics_city_population = update_column_name(dim_demographics_city_population, new_columns)
    dim_demographics_city_population = dim_demographics_city_population.\
    select(concat(dim_demographics_city_population.city,\
                  dim_demographics_city_population.state_code)\
           .alias('idx'),'city','state_code','male_pop','female_pop',\
           'num_vetarans','foreign_born','median_age',\
           'avg_household_size')
    dim_demographics_city_population = dim_demographics_city_population\
    .withColumn('city', upper(col('city')))
    dim_demographics_city_population = dim_demographics_city_population\
    .withColumn('idx', upper(col('idx')))
    dim_demographics_city_population.write.mode("overwrite").parquet\
    (path='demographics_data/dim_demographics_city_population')
dim_demographics_city_population.show(5)

+-------------+-----------+----------+--------+----------+------------+------------+----------+------------------+
|          idx|       city|state_code|male_pop|female_pop|num_vetarans|foreign_born|median_age|avg_household_size|
+-------------+-----------+----------+--------+----------+------------+------------+----------+------------------+
|    HAMPTONVA|    HAMPTON|        VA|   66214|     70240|       19638|        6204|      35.5|              2.48|
|   EL MONTECA|   EL MONTE|        CA|   57961|     58784|        1686|       59655|      37.3|              3.67|
| SOMERVILLEMA| SOMERVILLE|        MA|   41028|     39306|        2103|       22292|      31.0|              2.43|
|SPRINGFIELDIL|SPRINGFIELD|        IL|   55639|     62170|        7525|        4264|      38.8|              2.22|
|     TEMPLETX|     TEMPLE|        TX|   33512|     38785|        7333|        3968|      33.6|              2.59|
+-------------+-----------+----------+--------+----------+------------+---------

In [8]:
with open("I94_SAS_Labels_Descriptions.SAS") as f:
    contents = f.readlines()
    
country_code = {}
for countries in contents[10:298]:
    pair = countries.split('=')
    code, country = pair[0].strip(), pair[1].strip().strip("'")
    country_code[code] = country
 
country_code = spark.createDataFrame(country_code.items(), ['code', 'country'])
country_code.write.mode("overwrite").parquet(path='label_data/country_code')
country_code.show(10)

+----+---------------+
|code|        country|
+----+---------------+
| 236|    AFGHANISTAN|
| 101|        ALBANIA|
| 316|        ALGERIA|
| 102|        ANDORRA|
| 324|         ANGOLA|
| 529|       ANGUILLA|
| 518|ANTIGUA-BARBUDA|
| 687|     ARGENTINA |
| 151|        ARMENIA|
| 532|          ARUBA|
+----+---------------+
only showing top 10 rows



In [10]:
city_code = {}
for cities in contents[303:962]:
    pair = cities.split('=')
    code, city = pair[0].strip("\t").strip().strip("'"),\
    pair[1].strip('\t').strip().strip("''")
    city_code[code] = city

city_code = spark.createDataFrame(city_code.items(), ['abbr_code', 'city'])
city_code.write.mode("overwrite").parquet(path='label_data/city_code')
city_code.show(10)

+---------+--------------------+
|abbr_code|                city|
+---------+--------------------+
|      ANC|ANCHORAGE, AK    ...|
|      BAR|BAKER AAF - BAKER...|
|      DAC|DALTONS CACHE, AK...|
|      PIZ|DEW STATION PT LA...|
|      DTH|DUTCH HARBOR, AK ...|
|      EGL|EAGLE, AK        ...|
|      FRB|FAIRBANKS, AK    ...|
|      HOM|HOMER, AK        ...|
|      HYD|HYDER, AK        ...|
|      JUN|JUNEAU, AK       ...|
+---------+--------------------+
only showing top 10 rows



In [11]:
state_code = {}
for states in contents[982:1036]:
    pair = states.split('=')
    code, state = pair[0].strip('\t').strip("'"), pair[1].strip().strip("'")
    state_code[code] = state
    
state_code = spark.createDataFrame(state_code.items(), ['abbr_code', 'state'])    
state_code.write.mode("overwrite").parquet(path='label_data/state_code')
state_code.show(10)

+---------+-----------------+
|abbr_code|            state|
+---------+-----------------+
|       AK|           ALASKA|
|       AZ|          ARIZONA|
|       AR|         ARKANSAS|
|       CA|       CALIFORNIA|
|       CO|         COLORADO|
|       CT|      CONNECTICUT|
|       DE|         DELAWARE|
|       DC|DIST. OF COLUMBIA|
|       FL|          FLORIDA|
|       GA|          GEORGIA|
+---------+-----------------+
only showing top 10 rows



In [53]:
from pathlib import Path
from os import listdir
from os.path import isfile, join, isdir

s3_bucket='/home/workspace/'
s3_bucket_path=Path(s3_bucket)
dir_processed_data=['label_data','immigration_data','temperature_data','airport_data','demographics_data']
subdir_processed_data=[Path(s3_bucket + f) for f in dir_processed_data]

for file_dir in s3_bucket_path.iterdir():
    if file_dir.is_dir():
        if file_dir in subdir_processed_data:
            onlyfiles = [f for f in listdir(file_dir) if isdir(join(file_dir, f))]
            file_dir=str(file_dir)
            for f in onlyfiles:
                filepath=file_dir+'/'+f
                df = spark.read.parquet(filepath)
                print("Table: " + filepath.split('/')[-1])
                schema = df.printSchema()

Table: city_code
root
 |-- abbr_code: string (nullable = true)
 |-- city: string (nullable = true)

Table: country_code
root
 |-- code: string (nullable = true)
 |-- country: string (nullable = true)

Table: state_code
root
 |-- abbr_code: string (nullable = true)
 |-- state: string (nullable = true)

Table: dim_immigration_airline
root
 |-- cic_id: double (nullable = true)
 |-- airline: string (nullable = true)
 |-- admin_num: double (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- visa_type: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- immigration_airline_id: long (nullable = true)

Table: fact_immigration
root
 |-- cic_id: double (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- city_code: string (nullable = true)
 |-- arrival_date: date (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- mode: double (nullable = true)
 |-- country: string (nullable = true)
 |-- matflag: string (

In [54]:
from pathlib import Path
from os import listdir
from os.path import isfile, join, isdir

s3_bucket='/home/workspace/'
s3_bucket_path=Path(s3_bucket)
dir_processed_data=['label_data','immigration_data','temperature_data','airport_data','demographics_data']
subdir_processed_data=[Path(s3_bucket + f) for f in dir_processed_data]

for file_dir in s3_bucket_path.iterdir():
    if file_dir.is_dir():
        if file_dir in subdir_processed_data:
            onlyfiles = [f for f in listdir(file_dir) if isdir(join(file_dir, f))]
            file_dir=str(file_dir)
            for f in onlyfiles:
                filepath=file_dir+'/'+f
                df = spark.read.parquet(filepath)
                record_num=df.count()
                print("Table: " + filepath.split('/')[-1])
                if record_num <= 0:
                    raise ValueError("This table is empty!")
                else:
                    print(f"Total {record_num} records.")

Table: city_code
Total 659 records.
Table: country_code
Total 288 records.
Table: state_code
Total 54 records.
Table: dim_immigration_airline
Total 3096313 records.
Table: fact_immigration
Total 8126 records.
Table: dim_immigration_visa
Total 17 records.
Table: dim_immigration_entdepa
Total 14 records.
Table: dim_immigration_personal
Total 3096313 records.
Table: dim_immigration_entdepd
Total 13 records.
Table: dim_temperature_city
Total 257 records.
Table: dim_temperature_usa
Total 687004 records.
Table: dim_airport_type
Total 7 records.
Table: dim_airport_city_state_code
Total 55075 records.
Table: dim_airport
Total 55075 records.
Table: dim_demographics_city_population
Total 596 records.
Table: dim_demographics_race
Total 5 records.
Table: dim_demographics_city_race
Total 2891 records.


In [ ]:
print(156)